In [1]:
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist
from tensorflow.keras import Sequential
import tensorflow.keras as keras
from tensorflow.keras import callbacks
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
import cv2 as cv
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, LabelBinarizer
import numpy as np

from PIL import Image
from PIL import ImageFont
from PIL import ImageDraw

import glob
import random
import os


2021-12-02 22:09:29.219955: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-02 22:09:29.220046: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
def get_digits(number):
    digitList = ['1','2','3','4','5','6','7','8','9']
    fonts = []
    digits = []
    digit_images = []

    for font in glob.glob("fonts/*"): # Create Image for every font
        fonts.append(os.path.abspath(font))

    for _ in range(number):
        rand_digit = random.randint(0, len(digitList) - 1)
        rand_font = random.randint(0, len(fonts) - 1)
        font = ImageFont.truetype(fonts[rand_font], random.randint(40, 50))
        width, height = font.getsize(digitList[rand_digit])
        img_width, img_height = 64, 64
        img = Image.new('L', (img_width, img_height), color='white')
        d = ImageDraw.Draw(img)
        d.text(((img_width - width) / 2, (img_height - height) / 2), digitList[rand_digit], font=font, fill=0, align="center") # TO ALIGN CHARACTER IN CENTER
        np_img = np.array(img)
        # cv.imshow("", np_img)
        # cv.waitKey(0)
        digits.append(digitList[rand_digit])
        digit_images.append(np_img)
    return np.array(digit_images), np.array(digits)

X, y = get_digits(10000)

In [3]:
train_data_vanilla, test_data_vanilla, train_label_vanilla, test_label_vanilla = train_test_split(X, y, test_size=0.2, stratify=y)

train_data = train_data_vanilla.flatten().reshape(train_data_vanilla.shape[0], np.prod(train_data_vanilla.shape[1:]))
test_data = test_data_vanilla.flatten().reshape(test_data_vanilla.shape[0], np.prod(test_data_vanilla.shape[1:]))


#preprocessor_CNN = RobustScaler()
preprocessor_CNN = MinMaxScaler()
#preprocessor_CNN = StandardScaler()
train_data_CNN = preprocessor_CNN.fit_transform(train_data).reshape((train_data.shape[0],64 ,64, 1))
test_data_CNN = preprocessor_CNN.transform(test_data).reshape((test_data.shape[0], 64, 64, 1))
train_data_CNN = train_data.reshape((train_data.shape[0],64 ,64, 1))
test_data_CNN = test_data.reshape((test_data.shape[0], 64, 64, 1))

preprocessor_label = LabelBinarizer()
train_label_CNN = preprocessor_label.fit_transform(train_label_vanilla)
test_label_CNN = preprocessor_label.transform(test_label_vanilla)



In [5]:
neural_model = Sequential([
    layers.Input(shape=(64,64,1)),
    layers.Conv2D(32, kernel_size=5, padding="same", activation="relu"),
    layers.MaxPooling2D(pool_size=2, strides=2),
    layers.Dropout(0.2),
    layers.Conv2D(64, kernel_size=3, activation="relu"),
    layers.MaxPooling2D(pool_size=2, strides=2),
    layers.Dropout(0.3),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(9, activation="softmax")
])
early_stopping = callbacks.EarlyStopping(
    min_delta=0.01,
    patience=5,
    restore_best_weights=True
)
adam = keras.optimizers.Adam(
    learning_rate=0.001,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-7,
    amsgrad=False,
    name="Adam"
)
neural_model.compile(
    optimizer = adam,
    loss = "categorical_crossentropy",
    metrics=['accuracy']
)

datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.05,
    zoom_range=0.1,
    fill_mode='nearest'
)
datagen.fit(train_data_CNN)

In [9]:
result = neural_model.fit(
    datagen.flow(train_data_CNN, train_label_CNN, batch_size=64),
    validation_data=(test_data_CNN, test_label_CNN),
    steps_per_epoch= len(train_data_CNN) // 64,
    epochs = 5,
    #shuffle=True,
    #verbose=False,
    use_multiprocessing=True,
    # callbacks=[early_stopping]
)

Epoch 1/5
125/125 [==============================] - 33s 265ms/step - loss: 0.1125 - accuracy: 0.9647 - val_loss: 1.4004e-04 - val_accuracy: 1.0000

In [10]:
neural_model.save("digit_CNN/digit_cnn.h5")